# Script to augment image training data for U-net
Created: 08/2022
Creator: Emma Davis
### Methods
- Rotate 90 degrees
- Flip
- Inject noise
- Random brightness enhance/reduction


In [1]:
# IMPORTS
import PIL
from PIL import Image, ImageOps, ImageEnhance
import glob
from tensorflow.keras.preprocessing.image import load_img
from IPython.display import display
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import random
import numpy as np

2023-02-16 14:48:03.323704: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-16 14:48:03.323760: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
# SET LOCATION OF ORIGINAL IMAGES AND DIRECTORIES TO WRITE IMAGES TO ONCE AUGMENTED
# 'orig' is where you should put original images (in a folder 'microglia') and their masks (in a
# folder 'masks')
working_dir = "/home/my_project/"
os.chdir(working_dir)

orig_img_dir = "/training_data/orig/"
aug_img_dir = "/training_data/aug/"


In [15]:
# FUNCTION TO NOISE INJECT IMAGE VIA SALT PEPPER METHOD
def add_noise(image):
    amount = random.uniform(0.1,0.5)
    output = np.copy(np.array(image))
    # add salt
    nb_salt = np.ceil(amount * output.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(nb_salt)) for i in output.shape]
    output[coords] = 1
    # add pepper
    nb_pepper = np.ceil(amount* output.size * 0.5)
    coords = [np.random.randint(0, i - 1, int(nb_pepper)) for i in output.shape]
    output[coords] = 0
    return ImageOps.grayscale(Image.fromarray(output))


# FUNCTION TO TAKE MASK IMAGE (TRICOLOUR MAP) AND MAP VALUES FOR ML MASK FORMAT
def map_col(im):
    newimdata = []
    blackgs = (0, 0, 0)
    whitegs = (255, 255, 255)
    greygs = (114, 114, 114)
    
    blackmp = 1#(1, 1, 1)
    greymp = 2#(2, 2, 2)
    whitemp = 3#(3, 3, 3)
    for color in im.getdata():
        if (color == blackgs) or (color < 50):
            newimdata.append(blackmp)
        elif (color == whitegs) or (color > 200):
            newimdata.append(whitemp)
        else:
            newimdata.append(greymp)

    newim = Image.new(im.mode,im.size)
    newim.putdata(newimdata)
    return newim

# TODO: THIS FUNCTION NEEDS TO BE ABLE TO SHIFT MASK IN THE SAME WAY AS THE IMAGE
def shift_img(img, mask):
    # convert to numpy array
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # create image data augmentation generator
    datagen = ImageDataGenerator(width_shift_range=[-100,100])
    # prepare iterator
    it = datagen.flow(samples, batch_size=1)
    # generate samples and plot
    batch = it.next()
    imageA = batch[0].astype('uint8')
    batch = it.next()
    imageB = batch[0].astype('uint8')
    return imageA, imageB

# FUNCTION TO AUGMENT IMAGES AND THEIR MASKS AND SAVE IN A SEPARATE FOLDER FROM ORIGINAL DATA
def aug_imgs(orig_img_dir, orig_mask_dir=None):
    # GET PATHS OF ALL ORIGINAL IMAGES AND MASKS
    orig_imgs = sorted(glob.glob(orig_img_dir + '/*.png'))
    if orig_mask_dir != None:
        orig_masks = sorted(glob.glob(orig_mask_dir + '/*.png'))
    else:
        orig_masks = None
    
    # SET TWO LISTS, ONE FOR AUGMENTED IMAGES AND ONE FOR MASK IMAGES
    aug_imgs = []
    aug_masks = []
    
    # ITERATE OVER ORIGINAL IMAGES AND MASKS AND AUGMENT EACH, ADDING TO LISTS. EXPECTED TO GET
    # ROUGHLY 144 IMAGES FROM EACH ORIGINAL IMAGE-MASK
    if (orig_masks == None) or (len(orig_imgs) == len(orig_masks)):
        for i in range(len(orig_imgs)):

            img = load_img(orig_imgs[i])
            if orig_masks != None:
                mask = load_img(orig_masks[i]).convert('L')
                # SETUP: CONVERT MASK FROM TRIMAP TO ML MASK FORMAT
                mask = map_col(PIL.ImageOps.autocontrast(mask))
            
            # SET UP LISTS TO ADDED AUGMENTED IMAGES TO FOR THIS PARTICULAR ORIG IMAGE-MASK
            aug_img_working = []
            aug_mask_working = []
            
            
            # 1: ADD ORIGINAL TO LIST (X1)
            aug_img_working.append(img)
            if orig_masks != None:
                aug_mask_working.append(mask)
            
            # 2: ADD ROTATED TO LIST (+3)
            aug_img_working.append(img.rotate(90))
            if orig_masks != None:
                aug_mask_working.append(mask.rotate(90))
            
            aug_img_working.append(img.rotate(180))
            if orig_masks != None:
                aug_mask_working.append(mask.rotate(180))
            
            aug_img_working.append(img.rotate(270))
            if orig_masks != None:
                aug_mask_working.append(mask.rotate(270))
            
            # 3: ADD FLIPPED TO LIST (+8)
            aug_img_working.append(ImageOps.flip(img))
            aug_img_working.append(ImageOps.mirror(img))
            
            aug_img_working.append(ImageOps.flip(aug_img_working[1]))
            aug_img_working.append(ImageOps.mirror(aug_img_working[1]))
            
            aug_img_working.append(ImageOps.flip(aug_img_working[2]))
            aug_img_working.append(ImageOps.mirror(aug_img_working[2]))
            
            aug_img_working.append(ImageOps.flip(aug_img_working[3]))
            aug_img_working.append(ImageOps.mirror(aug_img_working[3]))
            
            if orig_masks != None:
                aug_mask_working.append(ImageOps.flip(mask))
                aug_mask_working.append(ImageOps.mirror(mask))
                
                aug_mask_working.append(ImageOps.flip(aug_mask_working[1]))
                aug_mask_working.append(ImageOps.mirror(aug_mask_working[1]))
                
                aug_mask_working.append(ImageOps.flip(aug_mask_working[2]))
                aug_mask_working.append(ImageOps.mirror(aug_mask_working[2]))
                
                aug_mask_working.append(ImageOps.flip(aug_mask_working[3]))
                aug_mask_working.append(ImageOps.mirror(aug_mask_working[3]))
            
            
            # 4: ADD NOISE INJECTION USING NOISE FUNCTION
            for i in range(len(aug_img_working)):
                aug_img_working.append(add_noise(aug_img_working[i]))
                if orig_masks != None:
                    aug_mask_working.append(aug_mask_working[i])
            
            # 5: ADD RANDOM BRIGHTNESS (ON ALL ABOVE ONE RANDOM BRIGHTNESS FOR EACH: +36)
            for i in range(len(aug_img_working)):
                factor = random.uniform(0.5,1.5)
                aug_img_working.append(ImageEnhance.Brightness(aug_img_working[i]).enhance(factor))
                if orig_masks != None:
                    aug_mask_working.append(aug_mask_working[i])
            
            # 6: ADD RANDOM ZOOM (ON ALL ABOVE ONE RANDOM ROOM FOR EACH: +72)
            
            # CONCATENATE AUGMENTED IMAGES LISTS TO MAIN LISTS AND LOOP
            aug_imgs.extend(aug_img_working)
            if orig_masks != None:
                aug_masks.extend(aug_mask_working)
            
    else:
        print("Different number of original images to masks - please make sure " +
              "original images and masks are correctly numbered and in pairs")
    return aug_imgs, aug_masks
    

In [16]:
# RUN AUGMENTATION ON GIVEN DIRECTORIES (MAKE SURE DIRECTORIES EXIST AND ARE EMPTY)
aug_imgs, aug_masks = aug_imgs(orig_img_dir, orig_mask_dir)
#aug_imgs, _ = aug_imgs(orig_img_dir)
print(len(aug_imgs))
# ITERATE OVER LISTS AND SAVE INTO RESPECTIVE DIRECTORIES
for i in range(len(aug_imgs)):
    aug_imgs[i].save(aug_img_dir + "/mglia_" + str(i).zfill(5) + ".png")
    aug_masks[i].save(aug_mask_dir + "/mask_" + str(i).zfill(5) + ".png")


/tmp/slurm_49331636/ipykernel_24910/1345642436.py:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output[coords] = 1
/tmp/slurm_49331636/ipykernel_24910/1345642436.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output[coords] = 0


4128
